In [1]:
import pandas as pd
import mxnet
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split


device = torch.device("cuda:0")
print(f"Using {device}")

2022-06-28 18:40:09.713521: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Using cuda:0


In [2]:
data = pd.read_pickle("../result/tokenized_test_data.pkl")

In [3]:
pred_data = data.iloc[950000:1050000, :]
#앞쪽 5만은 학습사용 뒤쪽 5만은 학습 사용하지 않음
pred_A = pred_data[:50000]
pred_B = pred_data[50000:]

#어떤것으로 테스트할까요?
pred_data = pred_A[['제목', '본문', '민원발생지', '접수기관']]
#pred_data = pred_B[['제목', '본문', '민원발생지', '접수기관']]
pred_data.reset_index(inplace = True, drop = True)
pred_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      50000 non-null  object
 1   본문      49995 non-null  object
 2   민원발생지   49992 non-null  object
 3   접수기관    50000 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [4]:
pred_data['본문'] = pred_data['본문'].str.strip()
pred_data['본문'] = pred_data['본문'].str.replace("\r\n", '')
pred_data.head(5)

/tmp/ipykernel_23495/2637551822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_data['본문'] = pred_data['본문'].str.strip()
/tmp/ipykernel_23495/2637551822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_data['본문'] = pred_data['본문'].str.replace("\r\n", '')


,제목,본문,민원발생지,접수기관
0,정부는 방관자 입니까? 즉시 위례 사업시행자 LH 감독하여 책임이행토록 요구하여 주...,정부는 방관자 입니까? 즉시 위례 사업시행자 LH 감독하여 책임이행토록 요구하여 주...,경기도,국토교통부
1,2AA-2007-0240879에 관하여 포상금 지급이 가능한지 문의,2AA-2007-0240879에 관하여 다른 조건이 다 충족되었을 경우포상금 지급이...,서울특별시,국세청
2,목감1중 설립.,목감1중 설립은 우리 목감 주민들에게는 매우 중대한 문제입니다.학구통합 반대 의견을...,경기도,경기도교육청
3,목감1중 설립.,목감1중 설립은 우리 목감 주민들에게는 매우 중대한 문제입니다.학구통합 반대 의견을...,경기도,경기도교육청
4,병역의무자 겸직금지 규정의 역사 조사,병역의무자의 겸직금지 규정은 못해도 케케 묶은 옛날 시대에 만들어져서 지금까지 전해...,경상남도,국방부


In [5]:
from konlpy.tag import Okt
okt = Okt()

okt.nouns("토크나이저 import가 잘 되었는지 확인합니다")

['토크', '나이', '저', '확인']

In [6]:
pred_data.iloc[0,0]

'정부는 방관자 입니까? 즉시 위례 사업시행자 LH 감독하여 책임이행토록 요구하여 주십시오.(국토교통부)  안녕하십니까? 위례신도시 광역교통개선대책이 나온지 어언 12년이 지났습니다. 그러나 현재까지도 제일 중'

In [7]:
if pred_B.iloc[0,0] == pred_data.iloc[0,0]:####
    import copy
    from tqdm import tqdm
    pred_data['token'] = copy.deepcopy(pred_data['본문'])
    for idx, text in enumerate(tqdm(pred_data['token'])):
        text = str(text)
        if len(text) > 3000:
            tok = "TOO LONG"
        else:
            tok = okt.nouns(text)
            if len(tok) < 512:
                pred_data['token'][idx] = tok
            else:
                pred_data['token'][idx] = "TOO LONG"

In [8]:
label_data = data[:1000000]

label_to_int = {}
for i, item in enumerate(label_data['접수기관'].unique()):
    label_to_int[item] = i

In [17]:
idx_list = []
for idx, item in enumerate(pred_data['접수기관']):
    if item in label_to_int.keys():
        idx_list.append(idx)

In [18]:
print(len(pred_data) , len(idx_list))

50000 50000


In [19]:
pred_data = pred_data.iloc[idx_list, :]

In [20]:
pred_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      50000 non-null  object
 1   본문      49995 non-null  object
 2   민원발생지   49992 non-null  object
 3   접수기관    50000 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [21]:
def department_to_int(x):
    try:
        return label_to_int[x]
    except:
        print('error')

In [22]:
pred_data.head(3)

,제목,본문,민원발생지,접수기관
0,정부는 방관자 입니까? 즉시 위례 사업시행자 LH 감독하여 책임이행토록 요구하여 주...,정부는 방관자 입니까? 즉시 위례 사업시행자 LH 감독하여 책임이행토록 요구하여 주...,경기도,국토교통부
1,2AA-2007-0240879에 관하여 포상금 지급이 가능한지 문의,2AA-2007-0240879에 관하여 다른 조건이 다 충족되었을 경우포상금 지급이...,서울특별시,국세청
2,목감1중 설립.,목감1중 설립은 우리 목감 주민들에게는 매우 중대한 문제입니다.학구통합 반대 의견을...,경기도,경기도교육청


In [24]:
pred_data

,제목,본문,민원발생지,접수기관
0,정부는 방관자 입니까? 즉시 위례 사업시행자 LH 감독하여 책임이행토록 요구하여 주...,정부는 방관자 입니까? 즉시 위례 사업시행자 LH 감독하여 책임이행토록 요구하여 주...,경기도,1
1,2AA-2007-0240879에 관하여 포상금 지급이 가능한지 문의,2AA-2007-0240879에 관하여 다른 조건이 다 충족되었을 경우포상금 지급이...,서울특별시,41
2,목감1중 설립.,목감1중 설립은 우리 목감 주민들에게는 매우 중대한 문제입니다.학구통합 반대 의견을...,경기도,59
3,목감1중 설립.,목감1중 설립은 우리 목감 주민들에게는 매우 중대한 문제입니다.학구통합 반대 의견을...,경기도,59
4,병역의무자 겸직금지 규정의 역사 조사,병역의무자의 겸직금지 규정은 못해도 케케 묶은 옛날 시대에 만들어져서 지금까지 전해...,경상남도,27
...,...,...,...,...
49995,[생활불편신고]장애인 전용구역 불법주차,&lt;장애인 전용구역 불법주차&gt;\n경상북도 경산시 정평동 130 부근\n\n...,경상북도,28
49996,(SPP-2003-3102022)횡단보도 불법 주정차 신고입니다.\n\n* 안전신문...,[민원 유입 경로]: 모바일\n[사고발생지역]울산광역시 중구 남외동 455-1\n횡...,울산광역시,263
49997,[생활불편신고]불법주차,&lt;불법주차&gt; 인도 불법주차\n경상남도 양산시 물금읍 물금리 392-6 부...,경상남도,5
49998,[생활불편신고]불법주차,&lt;불법주차&gt; 불법주차\n경기도 화성시 영천동 511-42 부근\n\n[☆...,경기도,36


In [23]:
pred_data['접수기관'] = pred_data['접수기관'].apply(lambda x : department_to_int(x))
pred_data = pred_data[['token', '접수기관']]

KeyError: "['token'] not in index"

In [ ]:
pred_data.head(3)

,token,접수기관
0,"[입, 주민, 의견서, 발신, 성남, 판교, 대장, 블럭, 예정자, 일동, 수신, ...",3
1,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경기도, 성남시, 중원구, 금...",8
2,"[문자, 신고, 화곡로, 앞, 삼거리, 교차로, 서울, 도, 불법, 주차, 신고]",49


### 여기는 class를 새로 만드는 부분이라서 다시 안해도됨

In [59]:
class BERTDataset(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(dataset.iloc[i]['token'])]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['접수기관']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



print('get bertmodel and vocab')
bertmodel, vocab = get_pytorch_kobert_model()


get bertmodel and vocab
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


### 여기는 해야함

In [19]:
max_grad_norm = 1
log_interval = 1000
warmup_ratio = 0.1
batch_size = 8
max_len = 512


print("data setting")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

pred_data_B = BERTDataset(pred_data, tok, max_len, True, False)

 
pred_dataloader = torch.utils.data.DataLoader(
    pred_data_B, batch_size = batch_size, num_workers = 8)


data setting
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
for i in pred_dataloader:
    print(i[0]) # input
    print(i[1]) # ?
    print(i[2]) # label
    print(i[3]) # ?
    break

tensor([[   2, 3836, 4228,  ...,    1,    1,    1],
        [   2, 2173, 3589,  ...,    1,    1,    1],
        [   2, 2123, 3012,  ...,    1,    1,    1],
        ...,
        [   2, 3892, 4438,  ...,    1,    1,    1],
        [   2, 1636, 2407,  ...,    1,    1,    1],
        [   2, 3552, 4213,  ...,    1,    1,    1]], dtype=torch.int32)
tensor([356, 260,  20,  44,  23, 188, 126,  23], dtype=torch.int32)
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)
tensor([  3,   8,  49, 115,  49,  61, 115,  49], dtype=torch.int32)


### 여기는 모델에 대한 부분이라서 안해도 됨

In [62]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(label_data['접수기관'].unique()),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
PATH = '../result/kobert/'

pred_model = torch.load(PATH + 'KoBERT_0627_py.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
# pred_model.load_state_dict(torch.load(PATH + 'Kobert_0627_py_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

# checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기
# pred_model.load_state_dict(checkpoint['model'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [55]:
# PATH = '../result/kobert/'

# pred_model = torch.load(PATH + 'KoBERT_0623_e5.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
# #pred_model.load_state_dict(torch.load(PATH + 'Kobert_0623_e5_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

In [45]:
model = BERTClassifier(pred_model).to(device)

In [63]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    #print(f"acc {train_acc} /  mi {max_indices} ==  {Y}")
    return train_acc

### Data Set만 준비한다면 여기를 계속 다른 인풋으로 돌려보면서 확인 가능

In [81]:
out_lst = []
pred_acc = 0
pred_model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(pred_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = pred_model(token_ids, valid_length, segment_ids)
    #out_lst.append(out)
    pred_acc += calc_accuracy(out, label)
    max_vals, max_indices = torch.max(out, 1)
    print(f"y = {label}, 'pred  ={max_vals}, {max_indices}, acc = {pred_acc}")
    if batch_id > 10:
        break
pred_acc / (batch_id+1)

  0%|          | 0/1764 [00:00<?, ?it/s]

y = tensor([  3,   8,  49, 115,  49,  61, 115,  49], device='cuda:1'), 'pred  =tensor([ 6.9040,  2.6896,  9.5187,  6.7664, 14.6903, -0.3631, 16.2148, -0.0783],
       device='cuda:1', grad_fn=<MaxBackward0>), tensor([  2,   6,   8,  16, 170,   6, 128,  26], device='cuda:1'), acc = 0.0
y = tensor([ 49,   1, 267,   1,  38,  28,  49,  42], device='cuda:1'), 'pred  =tensor([13.3809, 13.9972, 11.4588, 10.7679,  3.7729, 14.2709, -0.1922,  9.5290],
       device='cuda:1', grad_fn=<MaxBackward0>), tensor([  8,   1, 186,   1,   1,  18,  26, 102], device='cuda:1'), acc = 0.25
y = tensor([  1,  16,  49,  49,  36, 326,  49,  49], device='cuda:1'), 'pred  =tensor([13.7426, 12.7054,  7.3291, 14.9016,  3.8409, 15.7294, 13.3692,  0.7885],
       device='cuda:1', grad_fn=<MaxBackward0>), tensor([  1,   8,   8,  86,  26, 225,  35, 102], device='cuda:1'), acc = 0.375
y = tensor([ 49,  49, 234, 267,  49,  49,  72,  49], device='cuda:1'), 'pred  =tensor([11.0366,  5.1702, 12.7160, 10.8489,  7.7380, 14.3519

0.0625

왜 안되는걸까???? 학습에 사용한 데이터로 해보자

### 압축했으니까 밑부분은 안해도 됨

In [27]:
pred_B = pd.read_pickle("../result/sample5_tokenized.pkl")

In [28]:
pred_B

,제목,본문,민원발생지,접수기관,token
0,농산물(한방재) 부재료의 중량표시 방법,안녕하십니까! 수고많으십니다.농산물(부재료)에 대한 중량표시 및 오차범위에 대한 질...,전라북도,식품의약품안전처,"[안녕하십니까, 수고, 농산물, 부, 재료, 대한, 중량, 표시, 및, 오차, 범위..."
1,전문건설업간 하도급계약에 관련,안녕하세요 저는 서울에서 일반토목공사를 하는 현장소장입니다.다름이 아니라 전문공사 ...,경기도,국토교통부,"[저, 서울, 토목공사, 장소, 전문, 공사, 상하, 수도, 설비, 사업, 발주, ..."
2,녹색건축 예비인증 후 설계변경시 예비인증을 다시 받아야 하는 경우에 관한 질의,1. 녹색건축 예비인증을 받은 이후에 각각 아래 1)에서 4)와 같은 설계변경 또는...,경기도,국토교통부,"[녹색건축, 예비, 인증, 이후, 각각, 아래, 설계, 변경, 사업, 변경, 발생,..."
3,(SPP-1803-1600277)임대아파트의 에어컨 실외기 문제로 호소합니다. 저는 임,[민원 유입 경로]: 모바일\n[사고발생지역]대구광역시 동구 각산동 1062\n[기...,대구광역시,한국토지주택공사,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 대구광역시, 동구, 각산동, ..."
4,[생활불편신고]불법 주차,&lt;불법주정차&gt; 18-03-15 오전부터 24시간 이상 도로변에 불법주차되...,경상남도,경상남도 양산시,"[불법, 정차, 오전, 이상, 도로, 변, 불법, 주차, 오늘, 사진, 첨부, 신고..."
...,...,...,...,...,...
999995,(SPP-2103-1507152)우회전차선 진입 후 앞의 차의 주정차로 인해 교통흐름을,[민원 유입 경로]: 모바일\n[사고발생지역]경상남도 창원시 의창구 동읍 용잠리 2...,경상남도,경상남도 창원시,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경상남도, 창원시, 의창구, ..."
999996,억울한 운영정지 자격정지 처분에 대한 심의를 다시금 부탁드립니다,안녕하십니까 저는 남양주시 오남읍에 위치한 가정어린이집 원장입니다 작년 9월 어린이...,경기도,보건복지부,"[안녕하십니까, 저, 남양주시, 오남읍, 위치, 가정, 어린이집, 원장, 작년, 어..."
999997,가양1동 주민센터 남자직원분,3월15일 오후2시에서 3시사이에 4번창구에서 근무하던 남자분 이름을 차마 제가 못...,서울특별시,서울특별시,"[오후, 번, 창구, 근무, 남자, 이름, 차마, 제, 후회, 앞, 아주머니, 업무..."
999998,(SPP-2103-1507122)기타 불법주정차 불법 주정차 신고입니다.\n\n* ...,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 구미시 송정동 518\n기타 ...,경상북도,경상북도 구미시,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경상북도, 구미시, 송정동, ..."


In [29]:
pred_B['접수기관'] = pred_B['접수기관'].apply(lambda x : label_to_int[x])
pred_B = pred_B[['token', '접수기관']]

In [30]:
pred_B

,token,접수기관
0,"[안녕하십니까, 수고, 농산물, 부, 재료, 대한, 중량, 표시, 및, 오차, 범위...",0
1,"[저, 서울, 토목공사, 장소, 전문, 공사, 상하, 수도, 설비, 사업, 발주, ...",1
2,"[녹색건축, 예비, 인증, 이후, 각각, 아래, 설계, 변경, 사업, 변경, 발생,...",1
3,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 대구광역시, 동구, 각산동, ...",3
4,"[불법, 정차, 오전, 이상, 도로, 변, 불법, 주차, 오늘, 사진, 첨부, 신고...",5
...,...,...
999995,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경상남도, 창원시, 의창구, ...",6
999996,"[안녕하십니까, 저, 남양주시, 오남읍, 위치, 가정, 어린이집, 원장, 작년, 어...",58
999997,"[오후, 번, 창구, 근무, 남자, 이름, 차마, 제, 후회, 앞, 아주머니, 업무...",49
999998,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경상북도, 구미시, 송정동, ...",48


In [31]:
pred_B_data = BERTDataset(pred_B, tok, 512, True, False)

pred_B_dataloader = torch.utils.data.DataLoader(
    pred_B_data, batch_size = 4, num_workers = 8)

In [33]:
pd.to_pickle(pred_B_dataloader, "../data/dataloader.pkl")

In [48]:
for i in pred_B_dataloader:
    print(i)
    break

[tensor([[   2, 3135, 5724,  ...,    1,    1,    1],
        [   2, 3990, 2726,  ...,    1,    1,    1],
        [   2, 1495, 5384,  ...,    1,    1,    1],
        [   2, 2173, 3589,  ...,    1,    1,    1]], dtype=torch.int32), tensor([ 80, 125, 344, 297], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 1, 1, 3], dtype=torch.int32)]


In [71]:
 pred_B_dataloader

TypeError: 'DataLoader' object is not subscriptable

In [82]:
test_acc = 0

model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(pred_B_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = pred_model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
    max_vals, max_indices = torch.max(out, 1)
    print(f"y = {label}, 'pred  ={max_vals}, {max_indices}, acc = {test_acc}")
    if batch_id > 10:
        break
#print("epoch {} test acc {}".format(test_acc / (batch_id+1)))


  0%|          | 0/250000 [00:00<?, ?it/s]

y = tensor([0, 1, 1, 3], device='cuda:1'), 'pred  =tensor([15.9727, 13.7207, 13.5524, 11.1652], device='cuda:1',
       grad_fn=<MaxBackward0>), tensor([0, 1, 1, 2], device='cuda:1'), acc = 0.75
y = tensor([5, 6, 7, 8], device='cuda:1'), 'pred  =tensor([10.1925,  5.8444, 17.8014, -7.1268], device='cuda:1',
       grad_fn=<MaxBackward0>), tensor([8, 4, 5, 4], device='cuda:1'), acc = 0.75
y = tensor([ 9,  1, 10,  1], device='cuda:1'), 'pred  =tensor([16.6986,  9.9397, 11.5673, 13.4723], device='cuda:1',
       grad_fn=<MaxBackward0>), tensor([7, 1, 8, 1], device='cuda:1'), acc = 1.25
y = tensor([11, 12, 13, 16], device='cuda:1'), 'pred  =tensor([13.6380, 14.5499, 10.8289, 13.7587], device='cuda:1',
       grad_fn=<MaxBackward0>), tensor([ 46,  10,   1, 117], device='cuda:1'), acc = 1.25
y = tensor([17, 17, 10, 21], device='cuda:1'), 'pred  =tensor([10.3911, 10.3911,  1.6662,  7.1922], device='cuda:1',
       grad_fn=<MaxBackward0>), tensor([13, 13,  6, 14], device='cuda:1'), acc = 1.25
y

In [69]:
test_acc / batch_id

0.05866913308669133